In [1]:
# load datasets from kaggle
def get_data():
    !mkdir /kaggle
    !mkdir /kaggle/input
    !pip install -q kaggle
    from google.colab import files
    files.upload()
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 /root/.kaggle/kaggle.json

    !kaggle datasets download -d meraxes10/bd-models
    !unzip bd-models.zip -d /kaggle/input/bd-models > /dev/null
    !rm bd-models.zip    

    !!kaggle datasets download -d azathoth42/myanimelist
    !unzip myanimelist.zip -d /kaggle/input/myanimelist > /dev/null
    !rm myanimelist.zip

In [2]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2022-12-08 13:46:52--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  35.9MB/s    in 0.4s    

2022-12-08 13:46:53 (35.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [3]:
#get_data()

In [4]:
!pip install pyspark > /dev/null

In [5]:
import pyspark
from pyspark.sql import SparkSession

In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window

In [7]:
import pyspark.ml.feature
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, CountVectorizer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.stat import Summarizer

In [8]:
import numpy as np
import re
import gc
import time

In [9]:
SEED = 42

In [10]:
ROOT = '/kaggle/input/myanimelist/'

In [11]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

In [12]:
conf = SparkConf().set("spark.ui.port", "4050") \
                  .set('spark.executor.memory', '9G') \
                  .set('spark.driver.memory', '7G') \
                  .set('spark.sql.autoBroadcastJoinThreshold', '-1')

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 13:47:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
get_ipython().system_raw('./ngrok http 4050 &')

In [14]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [15]:
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://82ea-34-28-1-148.ngrok.io


Anime data pre-processing

In [16]:
anime_df = spark.read.format("csv").option("header", "true") \
                                   .option("headers", "true") \
                                   .option('escape','"') \
                                   .option("inferSchema", "true") \
                                   .load(ROOT + "AnimeList.csv", sep=',')

In [17]:
cols = ['title', 'title_english', 'title_japanese', 'title_synonyms', 'image_url', 'aired_string', 'background',
       'broadcast', 'related', 'opening_theme', 'ending_theme', 'studio']
anime_df = anime_df.drop(*cols)

In [18]:
cols = ['premiered', 'producer', 'licensor', 'rank']
anime_df = anime_df.drop(*cols)

In [19]:
cols = ['aired', 'duration', 'airing']
anime_df = anime_df.drop(*cols)

In [20]:
anime_df = anime_df.fillna('', subset=['genre'])

In [21]:
anime_df = anime_df.withColumn(
    'genre',
    split(regexp_replace('genre', ' ', ''), ',').cast("array<string>").alias("genre")
)

In [22]:
anime_df = anime_df.withColumn("episodes", anime_df.episodes.cast('float'))
anime_df = anime_df.withColumn("score", anime_df.score.cast('float'))
anime_df = anime_df.withColumn("scored_by", anime_df.scored_by.cast('float'))
anime_df = anime_df.withColumn("popularity", anime_df.popularity.cast('float'))
anime_df = anime_df.withColumn("members", anime_df.members.cast('float'))
anime_df = anime_df.withColumn("favorites", anime_df.favorites.cast('float'))

In [23]:
anime_df.show(5)

+--------+----+--------+--------+---------------+--------------------+-----+---------+----------+--------+---------+--------------------+
|anime_id|type|  source|episodes|         status|              rating|score|scored_by|popularity| members|favorites|               genre|
+--------+----+--------+--------+---------------+--------------------+-----+---------+----------+--------+---------+--------------------+
|   11013|  TV|   Manga|    12.0|Finished Airing|PG-13 - Teens 13 ...| 7.63| 139250.0|     231.0|283882.0|   2809.0|[Comedy, Supernat...|
|    2104|  TV|   Manga|    26.0|Finished Airing|PG-13 - Teens 13 ...| 7.89|  91206.0|     366.0|204003.0|   2579.0|[Comedy, Parody, ...|
|    5262|  TV|   Manga|    51.0|Finished Airing|       PG - Children| 7.55|  37129.0|    1173.0| 70127.0|    802.0|[Comedy, Magic, S...|
|     721|  TV|Original|    38.0|Finished Airing|PG-13 - Teens 13 ...| 8.21|  36501.0|     916.0| 93312.0|   3344.0|[Comedy, Drama, M...|
|   12365|  TV|   Manga|    25.0|F

In [24]:
anime_df = CountVectorizer(inputCol="genre", outputCol="genre_fv").fit(anime_df).transform(anime_df)
anime_df = anime_df.drop('genre')

In [25]:
categoricalColumns = ['type', 'source', 'status', 'rating']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [26]:
numericCols = ['episodes', 'score', 'scored_by', 'popularity', 'members', 'favorites', 'genre_fv'] 
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="item_feats_profile")
stages += [assembler]

In [27]:
@udf("array<integer>")
def indices(v):
    return v.indices.tolist()

In [28]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(anime_df)
anime_df = pipelineModel.transform(anime_df)

In [29]:
selectedCols = ['anime_id', 'item_feats_profile']
anime_df = anime_df.select(selectedCols)

In [30]:
anime_df.show(5)

+--------+--------------------+
|anime_id|  item_feats_profile|
+--------+--------------------+
|   11013|(79,[0,8,21,23,29...|
|    2104|(79,[0,8,21,23,29...|
|    5262|(79,[0,8,21,25,29...|
|     721|(79,[0,7,21,23,29...|
|   12365|(79,[0,8,21,23,29...|
+--------+--------------------+
only showing top 5 rows



User data pre-processing

In [31]:
user_df = spark.read.format("csv").option("header", "true") \
                                   .option("headers", "true") \
                                   .option('escape','"') \
                                   .option("inferSchema", "true") \
                                   .load(ROOT + "UserList.csv", sep=',')

In [32]:
user_df = user_df.filter(col('username').isNotNull())
user_df = user_df.filter(col('stats_episodes').isNotNull())

In [33]:
cols = ['location', 'access_rank', 'stats_mean_score', 'birth_date', 'gender']
user_df = user_df.drop(*cols)

In [34]:
cols = ['join_date', 'last_online']
user_df = user_df.drop(*cols)

In [35]:
user_df = user_df.withColumn("user_id", user_df.user_id.cast('float'))
user_df = user_df.withColumn("user_watching", user_df.user_watching.cast('float'))
user_df = user_df.withColumn("user_completed", user_df.user_completed.cast('float'))
user_df = user_df.withColumn("user_onhold", user_df.user_onhold.cast('float'))
user_df = user_df.withColumn("user_dropped", user_df.user_dropped.cast('float'))
user_df = user_df.withColumn("user_plantowatch", user_df.user_plantowatch.cast('float'))
user_df = user_df.withColumn("user_days_spent_watching", user_df.user_days_spent_watching.cast('float'))
user_df = user_df.withColumn("stats_rewatched", user_df.stats_rewatched.cast('float'))
user_df = user_df.withColumn("stats_episodes", user_df.stats_episodes.cast('float'))

In [36]:
user_df.show(5)

+----------------+---------+-------------+--------------+-----------+------------+----------------+------------------------+---------------+--------------+
|        username|  user_id|user_watching|user_completed|user_onhold|user_dropped|user_plantowatch|user_days_spent_watching|stats_rewatched|stats_episodes|
+----------------+---------+-------------+--------------+-----------+------------+----------------+------------------------+---------------+--------------+
|        karthiga|2255153.0|          3.0|          49.0|        1.0|         0.0|             0.0|                   55.31|            0.0|        3391.0|
|RedvelvetDaisuki|1897606.0|         61.0|         396.0|       39.0|         0.0|           206.0|                  118.07|           80.0|        7094.0|
|       Damonashu|  37326.0|         45.0|         195.0|       27.0|        25.0|            59.0|                    83.7|            6.0|        4936.0|
|           bskai| 228342.0|         25.0|         414.0|       

In [37]:
cols = ['user_watching', 'user_completed', 'user_onhold', 'user_dropped', 'user_plantowatch', 'user_days_spent_watching', 
        'stats_rewatched', 'stats_episodes']
assembler = VectorAssembler(inputCols=cols, outputCol="user_feats_profile")
stages = [assembler]

In [38]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(user_df)
user_df = pipelineModel.transform(user_df)

In [39]:
selectedCols = ['username', 'user_id', 'user_feats_profile']
user_df = user_df.select(selectedCols)

In [40]:
user_df.show(5)

+----------------+---------+--------------------+
|        username|  user_id|  user_feats_profile|
+----------------+---------+--------------------+
|        karthiga|2255153.0|[3.0,49.0,1.0,0.0...|
|RedvelvetDaisuki|1897606.0|[61.0,396.0,39.0,...|
|       Damonashu|  37326.0|[45.0,195.0,27.0,...|
|           bskai| 228342.0|[25.0,414.0,2.0,5...|
|       shuzzable|2347781.0|[36.0,72.0,16.0,2...|
+----------------+---------+--------------------+
only showing top 5 rows



User-anime data pre-processing

In [41]:
user_anime_df = spark.read.format("csv").option("header", "true").load(ROOT + "UserAnimeList.csv")

In [42]:
cols = ['my_watched_episodes', 'my_start_date', 'my_finish_date', 'my_status', 'my_rewatching', 
        'my_rewatching_ep', 'my_last_updated', 'my_tags']
user_anime_df = user_anime_df.drop(*cols)

In [43]:
user_anime_df = user_anime_df.withColumn("anime_id", user_anime_df.anime_id.cast('float'))
user_anime_df = user_anime_df.withColumn("my_score", user_anime_df.my_score.cast('float'))

In [44]:
user_anime_df = user_anime_df.na.drop()

In [45]:
user_anime_df = user_anime_df.filter(user_anime_df.my_score <= 10)

In [46]:
user_anime_df = user_anime_df.filter(user_anime_df.my_score != 0)

In [47]:
user_anime_df.show(5)

+--------+--------+--------+
|username|anime_id|my_score|
+--------+--------+--------+
|karthiga|    21.0|     9.0|
|karthiga|    59.0|     7.0|
|karthiga|    74.0|     7.0|
|karthiga|   120.0|     7.0|
|karthiga|   178.0|     7.0|
+--------+--------+--------+
only showing top 5 rows



In [48]:
user_anime_df = user_anime_df.join(user_df, 'username', how='left')

In [49]:
user_anime_df = user_anime_df.drop('username')

In [50]:
user_anime_df = user_anime_df.join(anime_df, 'anime_id', how='left')

In [51]:
user_anime_df = user_anime_df.na.drop()

In [52]:
assembler = VectorAssembler(inputCols=["user_feats_profile", "item_feats_profile"], outputCol="features")

In [53]:
user_anime_df = assembler.transform(user_anime_df)

In [54]:
user_anime_df.show(5)

+--------+--------+--------+--------------------+--------------------+--------------------+
|anime_id|my_score| user_id|  user_feats_profile|  item_feats_profile|            features|
+--------+--------+--------+--------------------+--------------------+--------------------+
|     5.0|     7.0|641117.0|[8.0,626.0,1.0,5....|(79,[2,7,21,27,29...|(87,[0,1,2,3,4,5,...|
|     5.0|    10.0|334191.0|[2.0,32.0,0.0,0.0...|(79,[2,7,21,27,29...|(87,[0,1,4,5,7,10...|
|     5.0|     8.0|181006.0|[12.0,103.0,8.0,0...|(79,[2,7,21,27,29...|(87,[0,1,2,4,5,7,...|
|     5.0|     9.0|535783.0|(8,[1,5,7],[250.0...|(79,[2,7,21,27,29...|(87,[1,5,7,10,15,...|
|     5.0|     8.0|487361.0|[7.0,682.0,3.0,22...|(79,[2,7,21,27,29...|(87,[0,1,2,3,4,5,...|
+--------+--------+--------+--------------------+--------------------+--------------------+
only showing top 5 rows



Split training, oof, test

In [55]:
(training, test) = user_anime_df.randomSplit([0.8, 0.2], seed=SEED)

In [56]:
(training, valid) = training.randomSplit([0.9, 0.1], seed=SEED)

Popularity based

In [57]:
avg_score_by_anime = training.groupBy('anime_id').agg(avg('my_score').alias('preds_0'))

In [58]:
avg_score = training.agg(avg('my_score').alias('overall_average'))

In [59]:
c = avg_score.collect()

In [60]:
valid = valid.join(avg_score_by_anime, 'anime_id', how='left')

In [61]:
valid = valid.fillna(c[0].overall_average, subset=['preds_0'])

In [62]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score", predictionCol="preds_0")
rmse = evaluator.evaluate(valid)

22/12/08 14:04:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:04:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:04:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:04:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:05:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:05:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:05:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:05:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:05:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:05:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:05:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:05:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:06:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:06:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:06:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:06:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:06:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:06:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


...

22/12/08 14:07:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:07:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:07:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:07:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:08:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:08:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:08:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:08:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:08:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:08:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:08:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:08:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [63]:
print("RMSE:" + str(rmse))

RMSE:1.5803892683548855


In [64]:
test = test.join(avg_score_by_anime, 'anime_id', how='left')

In [65]:
test = test.fillna(c[0].overall_average, subset=['preds_0'])

Biased User Model

In [66]:
avg_score_by_user = training.groupBy('user_id').agg(avg('my_score').alias('preds_1'))

In [67]:
valid = valid.join(avg_score_by_user, 'user_id', how='left')

In [68]:
valid = valid.fillna(c[0].overall_average, subset=['preds_1'])

In [69]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score", predictionCol="preds_1")
rmse = evaluator.evaluate(valid)

22/12/08 14:21:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:21:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:22:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:22:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:22:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:22:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:22:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:22:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:22:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


...

22/12/08 14:24:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:24:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:24:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:24:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:24:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:24:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:24:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:24:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:25:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:25:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:25:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:25:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:25:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:25:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:25:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:25:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 14:25:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:25:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:26:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 14:26:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [70]:
print("RMSE:" + str(rmse))

RMSE:1.4939422780918006


In [71]:
test = test.join(avg_score_by_user, 'user_id', how='left')

In [72]:
test = test.fillna(c[0].overall_average, subset=['preds_1'])

Random Forest

In [73]:
from pyspark.ml.regression import RandomForestRegressor, RandomForestRegressionModel

In [74]:
rf = RandomForestRegressor(featuresCol='features', labelCol='my_score', 
                           numTrees=5, maxMemoryInMB=1024,
                           subsamplingRate=0.1)

In [75]:
rf_model = rf.fit(training)

22/12/08 14:44:14 WARN MemoryStore: Not enough space to cache rdd_291_0 in memory! (computed 527.2 MiB so far)
22/12/08 14:44:14 WARN BlockManager: Persisting block rdd_291_0 to disk instead.


22/12/08 14:45:11 WARN MemoryStore: Not enough space to cache rdd_291_3 in memory! (computed 527.2 MiB so far)
22/12/08 14:45:11 WARN BlockManager: Persisting block rdd_291_3 to disk instead.


22/12/08 14:46:30 WARN MemoryStore: Not enough space to cache rdd_291_6 in memory! (computed 98.8 MiB so far)
22/12/08 14:46:30 WARN BlockManager: Persisting block rdd_291_6 to disk instead.


22/12/08 14:46:59 WARN MemoryStore: Not enough space to cache rdd_291_7 in memory! (computed 527.2 MiB so far)
22/12/08 14:46:59 WARN BlockManager: Persisting block rdd_291_7 to disk instead.


22/12/08 14:47:32 WARN MemoryStore: Not enough space to cache rdd_291_8 in memory! (computed 222.4 MiB so far)
22/12/08 14:47:32 WARN BlockManager: Persisting block rdd_291_8 to disk instead.


...

22/12/08 14:59:13 WARN MemoryStore: Not enough space to cache rdd_291_10 in memory! (computed 222.4 MiB so far)
22/12/08 14:59:14 WARN MemoryStore: Not enough space to cache rdd_291_11 in memory! (computed 148.3 MiB so far)


22/12/08 14:59:24 WARN MemoryStore: Not enough space to cache rdd_291_12 in memory! (computed 334.4 MiB so far)


22/12/08 14:59:24 WARN MemoryStore: Not enough space to cache rdd_291_13 in memory! (computed 41.6 MiB so far)


22/12/08 14:59:37 WARN MemoryStore: Not enough space to cache rdd_291_15 in memory! (computed 41.6 MiB so far)
22/12/08 14:59:37 WARN MemoryStore: Not enough space to cache rdd_291_14 in memory! (computed 334.4 MiB so far)


22/12/08 14:59:49 WARN MemoryStore: Not enough space to cache rdd_291_17 in memory! (computed 41.6 MiB so far)
22/12/08 14:59:49 WARN MemoryStore: Not enough space to cache rdd_291_16 in memory! (computed 334.4 MiB so far)


In [76]:
# rf_model = RandomForestRegressionModel.load('/kaggle/input/bd-models/rf_model')

In [77]:
valid = rf_model.transform(valid)

In [78]:
valid = valid.withColumnRenamed("prediction", "preds_2")

In [79]:
test = rf_model.transform(test)

In [80]:
test = test.withColumnRenamed("prediction", "preds_2")

In [81]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score", predictionCol="preds_2")
rmse = evaluator.evaluate(valid)

In [82]:
print("RMSE:" + str(rmse))

RMSE:1.5836486450873215


Linear Regression

In [83]:
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
from pyspark.ml.feature import StandardScaler, StandardScalerModel

In [84]:
scaler = StandardScaler(inputCol="features", 
                        outputCol="std_features",
                        withStd=True, withMean=True)

In [85]:
model_scaler = scaler.fit(training)

In [86]:
# model_scaler = StandardScalerModel.load('/kaggle/input/bd-models/scaler_model')

In [87]:
training = model_scaler.transform(training)
valid = model_scaler.transform(valid)
test = model_scaler.transform(test)

In [88]:
lm = LinearRegression(featuresCol='std_features', labelCol='my_score', maxIter=10)

In [89]:
lr_model = lm.fit(training)

22/12/08 15:33:43 WARN Instrumentation: [648181a8] regParam is zero, which might cause numerical instability and overfitting.


In [90]:
# lr_model = LinearRegressionModel.load('/kaggle/input/bd-models/lr_model')

In [91]:
valid = lr_model.transform(valid)
test = lr_model.transform(test)

In [92]:
valid = valid.withColumnRenamed("prediction", "preds_3")
test = test.withColumnRenamed("prediction", "preds_3")

In [93]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score", predictionCol="preds_3")
rmse = evaluator.evaluate(valid)

In [94]:
print("RMSE:" + str(rmse))

RMSE:1.569880223422923


ALS

In [95]:
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [96]:
als = ALS(userCol="user_id", itemCol="anime_id", ratingCol="my_score", coldStartStrategy="drop")

param_grid = ParamGridBuilder().addGrid(als.rank, [25]) \
                               .addGrid(als.regParam, [0.1, 0.15]) \
                               .addGrid(als.maxIter, [10]) \
                               .build()

In [97]:
tvs = TrainValidationSplit(estimator=als,
                           estimatorParamMaps=param_grid,
                           evaluator=RegressionEvaluator(metricName="rmse", labelCol="my_score", predictionCol="prediction"),
                           trainRatio=0.8)

In [98]:
maxIter = 10
regParam = 0.1
rank=25

In [99]:
als = ALS(maxIter=maxIter, regParam=regParam, rank=rank, 
          userCol="user_id", itemCol="anime_id", ratingCol="my_score",
          coldStartStrategy="drop")

In [100]:
als_model = als.fit(training)

In [101]:
# als_model = ALSModel.load('/kaggle/input/bd-models/als_model')

In [102]:
valid = als_model.transform(valid)
test = als_model.transform(test)

In [103]:
valid = valid.withColumnRenamed("prediction", "preds_4")
test = test.withColumnRenamed("prediction", "preds_4")

In [104]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score", predictionCol="preds_4")
rmse = evaluator.evaluate(valid)

In [105]:
print("RMSE:" + str(rmse))

RMSE:1.2200258317584438


In [106]:
userRecs = als_model.recommendForAllUsers(10)

In [107]:
userRecs.show(5, False)

+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                                                                                                                                      |
+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1      |[{37105, 11.355072}, {35035, 10.00731}, {35055, 10.00731}, {35058, 10.00731}, {31004, 9.936791}, {26293, 8.943113}, {33607, 8.81378}, {44, 8.792454}, {25065, 8.738379}, {5114, 8.728648}]           |
|44     |[{37105, 10.309638}, {820, 8.633801}, {2921, 8.26084}, {3297, 8.210474}, {35055, 8.09686}, {35035, 8.09686}, {35058, 8.09686}, {31004, 8.06693}, {32, 7.9826703

In [108]:
als_model.save('/kaggle/working/als_model')

Ensemble

In [109]:
als_model_2 = ALSModel.load('/kaggle/working/als_model')

In [110]:
!zip -r file.zip /kaggle/working/als_model


  adding: kaggle/working/als_model/ (stored 0%)
  adding: kaggle/working/als_model/metadata/ (stored 0%)
  adding: kaggle/working/als_model/metadata/part-00000 (deflated 39%)
  adding: kaggle/working/als_model/metadata/._SUCCESS.crc (stored 0%)
  adding: kaggle/working/als_model/metadata/_SUCCESS (stored 0%)
  adding: kaggle/working/als_model/metadata/.part-00000.crc (stored 0%)
  adding: kaggle/working/als_model/userFactors/ (stored 0%)
  adding: kaggle/working/als_model/userFactors/.part-00004-c89dcdbc-3778-4b99-aea7-94e938635e35-c000.snappy.parquet.crc (stored 0%)
  adding: kaggle/working/als_model/userFactors/.part-00002-c89dcdbc-3778-4b99-aea7-94e938635e35-c000.snappy.parquet.crc (stored 0%)
  adding: kaggle/working/als_model/userFactors/part-00001-c89dcdbc-3778-4b99-aea7-94e938635e35-c000.snappy.parquet (deflated 9%)
  adding: kaggle/working/als_model/userFactors/part-00002-c89dcdbc-3778-4b99-aea7-94e938635e35-c000.snappy.parquet (deflated 9%)
  adding: kaggle/working/als_model/u

In [111]:
!ls

__notebook__.ipynb  als_model  file.zip  ngrok	ngrok-stable-linux-amd64.zip


In [112]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip

In [113]:
!pwd

/kaggle/working


In [114]:
userRecs.show(5, False)

+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                                                                                                                                      |
+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1      |[{37105, 11.355072}, {35035, 10.00731}, {35055, 10.00731}, {35058, 10.00731}, {31004, 9.936791}, {26293, 8.943113}, {33607, 8.81378}, {44, 8.792454}, {25065, 8.738379}, {5114, 8.728648}]           |
|44     |[{37105, 10.309638}, {820, 8.633801}, {2921, 8.26084}, {3297, 8.210474}, {35055, 8.09686}, {35035, 8.09686}, {35058, 8.09686}, {31004, 8.06693}, {32, 7.9826703

In [115]:
alpha_0 = 0.1
alpha_1 = 0.1
alpha_2 = 0.1
alpha_3 = 0.1
alpha_4 = 0.6

In [116]:
valid = valid.fillna(c[0].overall_average)

In [117]:
valid = valid.withColumn('prediction', alpha_0*valid['preds_0'] + alpha_1*valid['preds_1'] + alpha_2*valid['preds_2'] + alpha_3*valid['preds_3'] + alpha_4*valid['preds_4'] )

In [118]:
valid = valid.select(['anime_id', 'user_id', 'my_score', 'prediction'])

In [119]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score", predictionCol="prediction")
rmse = evaluator.evaluate(valid)

22/12/08 20:56:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:56:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:56:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:56:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 20:56:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:56:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:56:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:56:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 20:57:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:57:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:57:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:57:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 20:57:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:57:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:57:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:57:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 20:58:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:58:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 20:58:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


...

22/12/08 21:03:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:04:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:04:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:04:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:04:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:04:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:04:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:04:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:04:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:04:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:05:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:05:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:05:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:05:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:05:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:05:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [120]:
print("RMSE:" + str(rmse))

RMSE:1.2641943435787215


Test

In [121]:
test = test.fillna(c[0].overall_average)

In [122]:
test = test.withColumn('prediction', alpha_0*test['preds_0'] + alpha_1*test['preds_1'] + alpha_2*test['preds_2'] + alpha_3*test['preds_3'] + alpha_4*test['preds_4'] )

In [123]:
test = test.select(['anime_id', 'user_id', 'my_score', 'prediction'])

In [124]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score", predictionCol="prediction")
rmse = evaluator.evaluate(test)

22/12/08 21:15:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:15:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:15:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:15:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:16:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:16:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:16:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:16:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:16:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:16:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:16:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:17:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:17:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:17:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:17:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:17:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:17:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:17:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:17:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:18:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


...

22/12/08 21:23:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:23:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:23:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:23:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:24:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:24:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:24:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:24:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:24:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:24:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:24:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:24:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:24:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:25:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/08 21:25:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/12/08 21:25:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [125]:
print("RMSE:" + str(rmse))

RMSE:1.2645565731487813
